In [1]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [22]:
hop_teaming = pd.read_csv('../hop_team-team-zeppelin/data/DocGraph_Hop_Teaming_2018.csv', 
                          nrows = 100)
hop_teaming.head()

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1508062167,1730166109,350,370,53.922,72.612
1,1508065640,1730166109,25,25,49.800,55.006
2,1508052093,1730166109,16,16,109.500,70.593
3,1508172545,1730166109,14,14,103.357,75.483
4,1508285131,1730166109,20,21,89.952,89.880


In [11]:
pd.set_option('display.max_columns', None)

In [16]:
pd.set_option('display.max_rows', None)

In [17]:
npi = pd.read_csv('npidata_pfile_20050523-20230212.csv', 
                          nrows = 1000)
npi.tail(100)

,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider Other Organization Name,Provider Other Organization Name Type Code,Provider Other Last Name,Provider Other First Name,Provider Other Middle Name,Provider Other Name Prefix Text,Provider Other Name Suffix Text,Provider Other Credential Text,Provider Other Last Name Type Code,Provider First Line Business Mailing Address,Provider Second Line Business Mailing Address,Provider Business Mailing Address City Name,Provider Business Mailing Address State Name,Provider Business Mailing Address Postal Code,Provider Business Mailing Address Country Code (If outside U.S.),Provider Business Mailing Address Telephone Number,Provider Business Mailing Address Fax Number,Provider First Line Business Practice Location Address,Provider Second Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Provider Business Practice Location Address Country Code (If outside U.S.),Provider Business Practice Location Address Telephone Number,Provider Business Practice Location Address Fax Number,Provider Enumeration Date,Last Update Date,NPI Deactivation Reason Code,NPI Deactivation Date,NPI Reactivation Date,Provider Gender Code,Authorized Official Last Name,Authorized Official First Name,Authorized Official Middle Name,Authorized Official Title or Position,Authorized Official Telephone Number,Healthcare Provider Taxonomy Code_1,Provider License Number_1,Provider License Number State Code_1,Healthcare Provider Primary Taxonomy Switch_1,Healthcare Provider Taxonomy Code_2,Provider License Number_2,Provider License Number State Code_2,Healthcare Provider Primary Taxonomy Switch_2,Healthcare Provider Taxonomy Code_3,Provider License Number_3,Provider License Number State Code_3,Healthcare Provider Primary Taxonomy Switch_3,Healthcare Provider Taxonomy Code_4,Provider License Number_4,Provider License Number State Code_4,Healthcare Provider Primary Taxonomy Switch_4,Healthcare Provider Taxonomy Code_5,Provider License Number_5,Provider License Number State Code_5,Healthcare Provider Primary Taxonomy Switch_5,Healthcare Provider Taxonomy Code_6,Provider License Number_6,Provider License Number State Code_6,Healthcare Provider Primary Taxonomy Switch_6,Healthcare Provider Taxonomy Code_7,Provider License Number_7,Provider License Number State Code_7,Healthcare Provider Primary Taxonomy Switch_7,Healthcare Provider Taxonomy Code_8,Provider License Number_8,Provider License Number State Code_8,Healthcare Provider Primary Taxonomy Switch_8,Healthcare Provider Taxonomy Code_9,Provider License Number_9,Provider License Number State Code_9,Healthcare Provider Primary Taxonomy Switch_9,Healthcare Provider Taxonomy Code_10,Provider License Number_10,Provider License Number State Code_10,Healthcare Provider Primary Taxonomy Switch_10,Healthcare Provider Taxonomy Code_11,Provider License Number_11,Provider License Number State Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Provider License Number_12,Provider License Number State Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Provider License Number_13,Provider License Number State Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Provider License Number_14,Provider License Number State Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Provider License Number_15,Provider License Number State Code_15,Healthcare Provider Primary Taxonomy Switch_15,Other Provider Identifier_1,Other Provider Identifier Type Code_1,Other Provider Ide

In [7]:
db = sqlite3.connect('hop_teaming.sqlite')

for chunk in tqdm(pd.read_csv('DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)): 
    chunk.to_sql('hop', db, if_exists = 'append', index = False) 

0it [00:00, ?it/s]

In [ ]:
db.execute('CREATE INDEX tencode_description ON calls(tencode_description)')

In [ ]:
db.close()

In [ ]:
for chunk in tqdm(pd.read_csv('../data/npidata_pfile_20050523-20230212.csv', 
                              usecols = [],
                              chunksize = 10000))

In [ ]:
db = sqlite3.connect('npi.sqlite')

for chunk in tqdm(pd.read_csv('DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)): 
    chunk.to_sql('npi', db, if_exists = 'append', index = False) 

In [34]:
taxonomy_switches = [f'Healthcare Provider Primary Taxonomy Switch_{i}' for i in range(1, 16)]
primary_taxonomy = [f'Healthcare Provider Taxonomy Code_{i}' for i in range(1, 16)]

mask = npi[taxonomy_switch_cols].eq('Y').any(axis=1)
filtered_cols = [col for col in primary_taxonomy_cols if npi.loc[mask, col].eq('Y').any()]

if not filtered_cols:
    print("No column with 'Y' value")
else:
    primary_taxonomy_col = filtered_cols[0]
    filtered_df = npi[npi[primary_taxonomy_col].eq('Y')]

No column with 'Y' value in associated 'Healthcare Provider Primary Taxonomy Switch' column.


In [43]:
Y_df = npi['Healthcare Provider Primary Taxonomy Switch_1'].eq('Y')
filtered_df = npi.loc[Y_df, ['Healthcare Provider Taxonomy Code_1']]

In [47]:
filtered_df

,Healthcare Provider Taxonomy Code_1
555,231H00000X
556,367500000X
557,1835P1200X
558,1223G0001X
559,207Q00000X
560,207P00000X
561,207N00000X
562,208000000X
563,174400000X
564,1223G0001X


In [ ]:
# Monica's loop
for index, row in npi_test.iterrows():
    for n in range(1, 16):
        if row['primary_taxonomy'] == '':
            x = str(n)
            if row[f'Healthcare Provider Primary Taxonomy Switch_{x}'] == 'Y':
                npi_test.loc[index, 'primary_taxonomy'] = npi_test.loc[index, f'Healthcare Provider Taxonomy Code_{x}']

In [48]:
def get_primary_taxonomy(row):
    for n in range(1, 16):
        if row['primary_taxonomy'] == '':
            x = str(n)
            if row[f'Healthcare Provider Primary Taxonomy Switch_{x}'] == 'Y':
                return row[f'Healthcare Provider Taxonomy Code_{x}']
    return row['primary_taxonomy']

npi['primary_taxonomy'] = npi.apply(get_primary_taxonomy, axis=1)

KeyError: 'primary_taxonomy'